In [1]:
%matplotlib inline
import os.path
import contextlib
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

import network_sim as netsim
import iv_analysis as iv

# deal with matplotlib bugs
def try_twice(fn):
    try:
        return fn()
    except np.linalg.LinAlgError:
        return fn()

## We want to simulate many networks with a relatively large number of variables, to test different ratios of observed to unobserved variables.

* 6 variables of interest
* observable ratios from 100 down to 5%

### Create parameter set

In [6]:
n_rois = 6
n_observed = np.linspace(n_rois, 20 * n_rois, 20)
n_regions = int(max(n_observed))
frac_observed = n_observed / n_regions

n_trials = 200
n_time = 500

iv_prob = 0.025
iv_gain = 0.3
spt = 5

### Do trials for each set of parameters

In [7]:
def seeded_sim_network(seed: np.uint32=None):
    """Simulate network with specific rng seed"""
    
    if seed is not None:
        assert np.isscalar(seed), 'Seed must be a scalar'
        np.random.seed(seed)
    
    con_mat = netsim.gen_con_mat(n_regions, net_type='smallworld')
    return netsim.sim_network(
        con_mat=con_mat,
        n_time=n_time,
        iv_prob=iv_prob,
        iv_gain=iv_gain,
        steps_per_time=spt
    )

result_file = '../data_sim/activations_sparse.npz'

if not os.path.exists(result_file):
    
    seeds = np.random.randint(0, 2 ** 32, dtype=np.uint32, size=n_trials)
    con_mat = np.ndarray((n_trials, n_regions, n_regions))
    log_act = np.ndarray((n_trials, n_regions, n_time))
    iv = np.ndarray((n_trials, n_regions, n_time))
    
    for i in range(n_trials):
        
        # generate network and simulate
        log_act[i], iv[i], con_mat[i] = seeded_sim_network(seeds[i])
        
    np.savez(result_file, seeds=seeds, con_mat=con_mat, log_act=log_act, iv=iv)

## Analysis

For each method, 